In [6]:
import os
import json
from pathlib import Path

from gretel_client.projects import get_project, search_projects, models, Project

In [ ]:
PRE_TRAIN_PROJECT = "gretel-client-project-pretrained"

FIXTURES = Path().resolve()

FIXTURES


In [3]:
project = get_project(name=PRE_TRAIN_PROJECT)

project.name

'gretel-client-project-pretrained'

In [4]:
def poll(model):
    for update in model.poll_logs_status():
        if update.transitioned:
            print(f"Status is {update.status}")
        for log in update.logs:
            msg = f"{log['ts']}  {log['msg']}"
            if log["ctx"]:
                msg += f"\n{json.dumps(log['ctx'], indent=4)}"
            print(msg)
        if update.error:
            print(f"\t{update.error}")

In [30]:
def build_synth_model(project: Project):
    model = project.create_model(model_config="synthetics/default")
    model.data_source = str(FIXTURES / "us-adult-income.csv")
    model.create(runner_mode=models.RunnerMode.CLOUD, upload_data_source=True)
    poll(model)
    return model

syn_model = build_synth_model(project)  # 60b8f345e00f682f45819019
syn_model.model_id

Status is created
Status is pending
Status is active
2021-06-03T15:23:42.588807Z  Starting synthetic model training
2021-06-03T15:23:42.590634Z  Loading training data
2021-06-03T15:23:42.823670Z  Training data loaded
{
    "record_count": 5000,
    "field_count": 15
}
2021-06-03T15:23:44.591039Z  Creating semantic validators and preparing training data
2021-06-03T15:23:49.850943Z  Beginning ML model training
2021-06-03T15:24:00.217853Z  Training epoch completed
{
    "epoch": 0,
    "accuracy": 0.19751952588558197,
    "loss": 4.548249244689941,
    "val_accuracy": 0.21039772033691406,
    "val_loss": 4.22438907623291,
    "batch": 0
}
2021-06-03T15:24:02.493268Z  Training epoch completed
{
    "epoch": 1,
    "accuracy": 0.2443242222070694,
    "loss": 3.7114768028259277,
    "val_accuracy": 0.37224432826042175,
    "val_loss": 2.5953729152679443,
    "batch": 0
}
2021-06-03T15:24:04.758279Z  Training epoch completed
{
    "epoch": 2,
    "accuracy": 0.6332890391349792,
    "loss": 1.

'60b8f345e00f682f45819019'

In [31]:
def build_xf_model(project: Project):
    model = project.create_model(model_config=FIXTURES / "transforms_config.yml")
    model.data_source = str(FIXTURES / "account-balances.csv")
    model.create(runner_mode=models.RunnerMode.CLOUD, upload_data_source=True)
    poll(model)
    return model

xf_model = build_xf_model(project)  # 60b8f50ae00f682f4581901d
xf_model.model_id

Status is created
Status is pending
Status is active
2021-06-03T15:28:23.290142Z  Starting transforms model training
2021-06-03T15:28:23.291090Z  Loading training data
2021-06-03T15:28:23.522714Z  Training data loaded
{
    "record_count": 302,
    "field_count": 9
}
2021-06-03T15:28:23.523107Z  Beginning transforms model training
2021-06-03T15:29:59.313081Z  Saving model archive
2021-06-03T15:29:59.332709Z  Saving training report
2021-06-03T15:29:59.335146Z  Generating data preview
{
    "num_records": 100
}
2021-06-03T15:29:59.382641Z  Done generating data preview
2021-06-03T15:29:59.383247Z  Uploading artifacts to Gretel Cloud
2021-06-03T15:29:59.941579Z  Model creation complete!


'60b8f50ae00f682f4581901d'

In [9]:
def build_classify_model(project: Project):
    model = project.create_model(model_config=FIXTURES / "classify_config.yml")
    model.data_source = str(FIXTURES / "account-balances.csv")
    model.create(runner_mode=models.RunnerMode.CLOUD, upload_data_source=True)
    poll(model)
    return model

classify_model = build_classify_model(project)  # 60cd03dec34d2b6362538133
classify_model.model_id

Status is created
Status is pending
Status is active
2021-06-18T20:37:01.423274Z  Starting classify model creation
2021-06-18T20:37:01.424185Z  Loading validation dataset
2021-06-18T20:37:01.659229Z  Using first 100 records out of 302 for validation
2021-06-18T20:37:01.659578Z  Validation dataset loaded
{
    "record_count": 100,
    "field_count": 9
}
2021-06-18T20:37:01.659877Z  Beginning classify model creation
2021-06-18T20:37:07.418038Z  Saving model archive
2021-06-18T20:37:07.419747Z  Saving classify report
2021-06-18T20:37:07.421057Z  Saving classify results preview
2021-06-18T20:37:07.422747Z  Uploading artifacts to Gretel Cloud
2021-06-18T20:37:07.921728Z  Model creation complete!


'60cd03dec34d2b6362538133'